<div class="alert alert-info"> <b>Комментарии студента:</b> 

Ссылка на дашборд:
https://public.tableau.com/app/profile/anastasia.peresypkina/viz/_17010185085250/Dashboard1?publish=yes
</div>

<div style="border:solid green 2px; padding: 20px">

Описание проекта.
    
Задача: провести анализ оттока клиентов «Метанпромбанк»:
1) Спрогнозировать вероятность оттока (на уровне следующего месяца) для каждого клиента;
2) Сформировать типичные портреты клиентов: выделить несколько наиболее ярких групп и охарактеризовать их основные свойства;
3) Проанализировать основные признаки, наиболее сильно влияющие на отток;
4) Сформулировать основные выводы и разработать рекомендации по повышению качества работы с клиентами:
    
    4.1. Выделить целевые группы клиентов;
    
    4.2. Предложить меры по снижению оттока;
    
    4.3 Определить другие особенности взаимодействия с клиентами.

Описание датасета bank_scrooge:
- `USERID` — идентификатор пользователя,
- `score` — баллы кредитного скоринга,
- `city` — город,
- `gender` — пол,
- `age` — возраст,
- `equity` — количество баллов собственности
- `balance` — баланс на счёте,
- `products` — количество продуктов, которыми пользуется клиент,
- `credit_card` — есть ли кредитная карта,
- `last_activity` — активный клиент,
- `EST_SALARY` — оценочный доход клиента,
- `сhurn` — признак оттока.ак оттока.

In [1]:
#импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import math
from scipy import stats as st
from scipy.stats import shapiro # для проведения теста Шапиро - Уилка
from scipy.stats import mannwhitneyu # для проведения теста Манна - Уитни
from statsmodels.stats.proportion import proportions_ztest # для проведения z-теста
from scipy. stats import lognorm
import statsmodels.api as sm

## Предобработка данных

In [2]:
#открыть датасет (bank_scrooge)
try:
    bank_scrooge = pd.read_csv('/datasets/bank_scrooge.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/bank_scrooge.csv')

HTTPError: HTTP Error 404: Not Found

### Проверка корректности наименования столбцов

In [ ]:
#ознакомиться с bank_scrooge
bank_scrooge.head()

In [ ]:
#уйдем от "научной нотации" для отображения числовых значений
pd.options.display.float_format = '{:.3f}'.format 

In [ ]:
# приведем наименования столбцов к единому формату
bank_scrooge.rename(columns = {'USERID':'user_id', 'EST_SALARY':'est_salary'}, inplace = True )

### Типы данных и обработка пропусков

In [ ]:
#общая информация о bank_scrooge
bank_scrooge.info()

In [ ]:
#количество впропусков в записях
bank_scrooge.isnull().sum()

In [ ]:
#визуализация пропусков

plt.figure(figsize=(10, 8))
sns.heatmap(bank_scrooge.isna(), yticklabels=False, cbar=False, cmap="YlGnBu")

plt.xlabel('Признак')  # подпись оси х
plt.ylabel('Кол-во пропусков')  # подпись оси у
plt.title('Распределение пропущенных значений')  # подпись названия

plt.show()

In [ ]:
#цикл для отображения доли пропусков
for col in bank_scrooge.columns:
    pct_missing = np.mean(bank_scrooge[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

In [ ]:
#посмотрим на пропуски непосредственно
bank_scrooge[bank_scrooge['age'].isna()]['age'].unique()

**Вывод:** доля пропусков крайне мала - удалим пропуски в столбце возраст ('age')

In [ ]:
# удаление пропусков в столбце возраст ('age'), так как доля пропусков крайне мала 
bank_scrooge.dropna(axis = 'index', subset = ['age'], inplace = True)

# убедимся, что действительно не осталось пропусков
bank_scrooge ['age'].isna().sum()

In [ ]:
# приведем тип данных столбца возраст к типу int64
bank_scrooge['age'] = bank_scrooge['age'].astype(int) 

In [ ]:
#попробуем понять причину отсутствия данных о балансе на счету (случайно ли упущены данные?) 
# сгруппируем по столбцу 'balance', где True - отсутствуют данные; False - значение заполнено
bank_scrooge.groupby(bank_scrooge['balance'].isnull()).mean().style.background_gradient(axis = 1)

**Вывод:**
наиболее сильная разница по параметрам: 
- кол-во баллов собственности;
- кол-во используемых клиентом продуктов банка: клиенты, в записях которых содержатся пропуска пользуются меньшим числом продуктов банка;
- оценочный доход клиента: клиенты, в записях которых содержатся пропуска имеют чуть более высокий уровень заработной платы;

In [ ]:
# посмотрим связаны ли пропуски в столбце balance с фактом оттока:
bank_scrooge[bank_scrooge['balance'].isna()]['churn'].value_counts()

**Вывод:**
В 12 случаях пропуск мог быть связан с тем, что клиент ушел из банка.

In [ ]:
#посмотрим на пропуски непосредственно
bank_scrooge[bank_scrooge['balance'].isna()]['balance'].unique()

**Вывод:**
Имеющиеся пропуски  - NaN. Данный тип значений относится к вещественным числам, float. NaN можно использовать в математических операциях, не вызывая ошибку из-за несовместимых типов.
Так как доля таких пропусков во всем датасете значительна (23%), удалить данные записи нельзя. Заполнить их каким-либо расчетным значением тоже не корректно. Следует также отметить, что сложно себе представить ситуацию, когда у банка нет данных о балансе (остатке денежных средств на счету клиента).
По моему мнению они относятся к категории MNAR (Missing Not At Random / Отсутствует не случайно). Учитывая вышеизложенное нужно оставить пропуски как есть.  

### Поиск дубликатов

In [ ]:
# поиск дубликатов
print('Дубликатов:', bank_scrooge.duplicated().sum())

In [ ]:
# поиск дубликатов по столбцам с уникальными значениями
print('Дубликатов по столбцу идентификатор пользователя:', bank_scrooge['user_id'].duplicated().sum())

In [ ]:
bank_scrooge['user_id'].value_counts()

In [ ]:
duplicateRows = bank_scrooge[bank_scrooge.duplicated(['user_id'])]
duplicateRows

In [ ]:
# посмотрим как распределены эти записи по городам присутствия банка
duplicateRows.groupby(['city', 'est_salary']).count()

**Вывод:** 
1) дубликаты user_id есть только в 2х городах (Рябинск, Ярославль) из 3х (нет в Ростове). При этом значения в столбце оценочный доход клиента ('est_salary') не дублируются, то есть скорее всего клиенты разные.

2) в банковской системе бывает, что у разных филиалов разные базы данных, в таком случае вполне возможно дублирование идентификатора пользователя ('user_id').

Резюмируя считаю, что можно оставить данные записи, так как для результат анализа - выделение сегмента, группы клиентов.

In [ ]:
bank_scrooge.drop('user_id', axis= 1 , inplace= True )

In [ ]:
#изучение данных в столбце 'city'; проверка на наличие дубликатов
bank_scrooge['city'].value_counts()

**Вывод:**
3 города, как и должно быть

In [ ]:
#изучение данных в столбце 'gender'; проверка на наличие дубликатов
bank_scrooge['gender'].value_counts()

**Вывод:**
2 пола, как и должно быть

In [ ]:
#изучение данных в столбце 'credit_card'; проверка на наличие дубликатов
bank_scrooge['credit_card'].value_counts()

In [ ]:
#изучение данных в столбце 'last_activity'; проверка на наличие дубликатов
bank_scrooge['last_activity'].value_counts()

In [ ]:
#изучение данных в столбце 'churn'; проверка на наличие дубликатов
bank_scrooge['churn'].value_counts()

<div style="border:solid green 2px; padding: 20px">
    
**Выводы по этапу Предобратока данных:**
1) для единообразия названия столбцов приведены к нижнему регистру;
    
2) не обнаружено полностью идентичных записей, однако, обнаружены дубликаты в столбце идентификатор пользователя ('user_id'). Принято решение оставить данные записи;
    
3) типы данных столбцов, в целом, корректны. Данные в столбце возраст приведены к типу int32, так как нас интересует полное кол-во лет и чаще всего именно полное кол-во лет отображается (без .0).
    
4) общий объем датасета - 10 000 строк. Из них пропуски есть в 2х столбцах:
   - возраст (26 пропущенных значений);
   - баланс на счете (2 295 пропущенных значений);
    
5) удалены строки с пропуском в столбце 'возраст';
    
6) пропуски в столбце 'баланс на счете' оставлены без изменений: определено, что данные пропуски это Nan значения, которые можно использовать в математических операциях. Также крайне не понятно как в банке может не быть информации о балансе клиента (то есть не ясна природа этих пропусков и заполнять их каким-либо расчетными значениями неверно). По моему мнению они относятся к категории MNAR (Missing Not At Random / Отсутствует не случайно).   
    
</div>

## Исследовательский анализ данных

In [ ]:
#визуализация распределения признаков, общее знакомство
bank_scrooge.hist(figsize=(20,12));

**Выводы:**
- `score` — баллы кредитного скоринга: скошенное распределение (отрицательная ассиметрия)
- `age` — возраст: смещение вправо, положительная ассиметрия
- `equity` — количество баллов собственности: 2 пика: чаще встречается 0 оценка и далее уже росто от 2х баллов.
- `balance` — баланс на счёте: мы знаем, что есть 23% пропусков (Nan) и, очевидно, много значений баланса клиентов близких к 0.
- `products` — количество продуктов, которыми пользуется клиент: 5 типов продуктов, с учетом удаленной записи с 0 значением - 4ре пика, ожидаемо.
- `credit_card` — есть ли кредитная карта:бинарные данные, отсюда просто 2 пика. ожидаемо.
- `last_activity` — активный клиент:бинарные данные, отсюда просто 2 пика. ожидаемо.
- `est_salary` — оценочный доход клиента: смещение вправо, положительная ассиметрия. Чаще встречается низкий уровень оценочного дохода клиента, что ожидаемо.
- `сhurn` — признак оттока: бинарные данные, отсюда просто 2 пика. ожидаемо.

In [ ]:
# изучение данных
bank_scrooge.describe()

**Выводы:**
1) в столбце products минимальное значение  = 0, то есть клиент банка не имеет ни одного продукта этого банка. А такое невозможно, необходимо понять причину такого минимального значения. При этом максимально кол-во банковских продуктов клиента = 5;

2) клиенты банка люди в возрасте от 18 до 86 лет; Средний возраст составляет 42 года

3) средний показатель в столбце оценочный доход клиента ('est_salary') составляет 147 866,88, при этом максимальное значение 1 395 064.45 - вероятно тут есть аномалии/выбросы (мы рассматриваем региональный банк, там маловероятны клиенты с таким уровнем дохода); нужно посмотреть, возможно искл. из датасета.

4) средний показатель в столбце баланс на счёте ('balance') составляет 827 794.307, при этом максимальное значение 119 113 552.010. Возможно среди клиентов есть очень состоятельные люди, но все равно есть смысл посмотреть внимательнее.	

In [ ]:
# посмотрим на стороку, где значение в столбце 'products' = 0
bank_scrooge[bank_scrooge['products']==0]

Видимо данный клиент уже не является активным, так как last_activity имеет значение 0.

In [ ]:
#удаление данных, где значение в графе количество продуктов, которыми пользуется клиент ('priducts') = 0
bank_scrooge = bank_scrooge[bank_scrooge['products']!=0]

### Выбросы, аномальные значения

In [ ]:
#цикл для расчет IQR и пределов/границ выбросов
for x in ['score', 'balance', 'est_salary' ]:
    IQR = st.iqr(bank_scrooge[x], interpolation = 'midpoint', nan_policy='omit')
    Q1 = np.percentile(bank_scrooge[x], 25 , interpolation = 'midpoint' )
    Q3 = np.percentile(bank_scrooge[x], 75 , interpolation = 'midpoint' )
    low_lim = Q1 - 1.5 * IQR
    up_lim = Q3 + 1.5 * IQR
    print(x, ' '*(15-len(x)),  'IQR =', IQR)
    print(' '*15, 'нижняя граница =', low_lim)
    print(' '*15, 'верхняя граница =', up_lim)
    print()

**Вывод**: все значения набора данных, которые отклоняются от медианы больше, чем на 3IQR, считаются выбросами

In [ ]:
# посмотрим строки, где значение оценочного дохода клиента 'est_salary' > 3*323 374.625 + 147 866.886 = 1 117 990,761
bank_scrooge.query('est_salary > 1117990')

In [ ]:
#сгруппируем по количеству баллов собственности ('equity')
bank_scrooge.query('est_salary > 1117990').groupby(['equity']).count()

**Вывод:**
Нет доверия к сотрудникам банка «Метанпром» в городе Ярославль:
из 24х записей о клиентах, чей доход более 1 117 990,00 у 14ти записей нет данных о балансе, количество баллов собственности ('equity') = 0 (то есть нет информации о собственности клиента); всего лишь 1 банковский продукт чаще всего. 

Все это ... странно. 

In [ ]:
pd.pivot_table(bank_scrooge[['credit_card', 'equity', 'last_activity', 'city', 'products', 'churn']],
               index = ['city', 'products'],
               values=['churn'],
               aggfunc = [np.mean] # узнаем среднее значение
              ).style.background_gradient(axis = 1) # форматирование

### Портрет клиента

In [ ]:
# Создадим сводную таблицу для отображение мер центральной тенденции по некоторым из параметров
table_1 = pd.pivot_table(bank_scrooge, 
                           values=['balance','est_salary', 'score','age','equity','products','city', 'gender'],
                           index=['churn'],
                           aggfunc={'balance': 'mean', 'est_salary':'mean',
                                    'score': 'mean', 'age':'mean',
                                    'equity':np.median, 'products':np.median}).T
table_1['% (delta)'] = ((table_1[1]/table_1[0]) - 1) * 100

In [ ]:
# Создадим сводную таблицу для категориальных признаков
table_mode = pd.pivot_table(bank_scrooge, values=['city', 'gender','last_activity','credit_card'], 
                index=['churn'], aggfunc={
                'city':st.mode, 'gender':st.mode,
                'last_activity':st.mode, 'credit_card':st.mode}).T
table_mode['% (delta)'] = 0

In [ ]:
pd.concat([table_1,table_mode])

### Кодировка данных (город, пол)

In [ ]:
bank_scrooge_dummy = bank_scrooge.copy()
bank_scrooge_dummy['gender_origin'] = bank_scrooge_dummy['gender'] # оставим этот слобец он далее понадобится
# применим функцию к столбцу пол ('gender')
bank_scrooge_dummy = pd.get_dummies(bank_scrooge_dummy, columns = ['gender'],
               prefix = '',
               prefix_sep = '')

In [ ]:
# полностью предобработанные данные запишем в переменную df 
df = bank_scrooge_dummy.copy()
df['city_origin'] = df['city'] # оставим этот слобец он далее понадобится

# применим функцию к столбцу город ('city')
df = pd.get_dummies(df, columns = ['city'],
                    prefix = '',
                    prefix_sep = '',
                    drop_first = False)

# приведем наименования столбцов к единому формату
df.rename(columns = {'Ростов':'rostov', 'Рыбинск':'rubinsk', 'Ярославль':'yaroslavl', 'Ж':'female', 'М':'male'}, inplace = True )

In [ ]:
# проверка
df.head()

### Распределение признаков

In [ ]:
# посчитаем % оттока от числа клиентов по городам
cities = bank_scrooge_dummy.groupby('city')\
  .agg({'churn': 'mean'})\
  .sort_values(by='churn', ascending=False)\
  .rename(columns={'churn': '% оттока'})\
  .style.format({'% оттока': '{:.2%}'})\

cities

**Вывод:**
Больше всего ушло клиентов из Ярославля (19,02%); Ростов (18,77%); Рыбинск (16,18%)

In [ ]:
#доля оттока по продукту через pivot_table()
outflow_share = bank_scrooge_dummy.pivot_table(index='products',
                                               values='churn',
                                               aggfunc=['count', 'sum', 'mean']).sort_values(by=('mean', 'churn'))  # сортировка по % оттока
outflow_share.columns = ['Общее количество клиентов', 'Количество ушедших клиентов', '% оттока']
outflow_share_formatted = outflow_share.style.format({'% оттока': '{:.2%}'})  # применение форматирования
outflow_share_formatted

**Вывод:**
В порядке убывания:
- 4ре продукта - отток 63,29%;
- 5 продуктов - отток 42,11%;
- 3 продукта - 28,59%
- 2 продукта - 19,11%
- 1 продукт - 7,06%

In [ ]:
#доля оттока по количествe баллов собственности через pivot_table()
equity_share = bank_scrooge_dummy.pivot_table(index = 'equity', 
                               values = 'churn',
                               aggfunc = ['count', 'sum', 'mean']).sort_values(by=('mean', 'churn')) # сортировка по % оттока
equity_share.columns = ['Общее количество клиентов', 'Количество ушедших клиентов', '% оттока']
equity_share_formatted = equity_share.style.format({'% оттока': '{:.2%}'})  # применение форматирования
equity_share_formatted

**Вывод:** % оттока растет по мере увеличения кол-ва баллов собственности, однако, следует отметить, что и кол-во клиентов значимо сокращается начиная с 8ми баллов собственности. Отсюда и больший %.

Посмотрим распределение признаков

In [ ]:
# список с бинарными значениями
features = [
    'products', 'credit_card', 'last_activity', 'churn', 'female', 'male', 'rostov', 'rubinsk', 'yaroslavl' 
    ]

In [ ]:
# визуализация
row, col, num_cols = 0, 0, 3
fig, myplot = plt.subplots(figsize = (15,8), nrows = math.ceil(len(features) / num_cols), ncols = num_cols)

for u in features:
    sns.barplot(
        x = df[u].value_counts().index, 
        y = df[u].value_counts(), 
        ax  = myplot[row, col]
    )
    myplot[row, col].set_xlabel("")
    myplot[row, col].set_title(u, fontsize = 15)
    myplot[row, col].set_ylabel("Количество")
    col = col + 1
    if col == num_cols:
        col = 0
        row = row + 1

plt.subplots_adjust(hspace = 0.5)
plt.subplots_adjust(wspace = 0.3)

for v in range(row):
    for z in range(num_cols):
        for patch in myplot[v,z].patches:
            label_x = patch.get_x() + patch.get_width()/2  
            label_y = patch.get_y() + patch.get_height()/2
            myplot[v,z].text(label_x, label_y, 
                             str(int(patch.get_height())) + '('+'{:.0%}'.format(
                               patch.get_height()/len(df))+')',
                            horizontalalignment='center', verticalalignment='center')

**Вывод:**
1) по количеству банковских продуктов:(top3)
   - 51% клиентов имеет 2 банковских продукта;
   - 33% клиентов имеет 1 банковский продукт;
   - 10% клиентов имеет 3 банковских продукта;
   
2) кредитная карта есть у 68% клиентов;

3) активность клиентов:
  - 52% активны;
  - 48% не активны;
  
4) **в отток ушло 18% клиентов**;

5) гендерное распределение среди клиентов 50/50;

6) распределение по городам: Ростов (14%), Рыбинск (27%), Ярославль (59%)

### Столбчатые гистограммы и распределения признаков для тех, кто ушёл (отток) и тех, кто остался

In [ ]:
# список 
features_1 = [
    'products', 'credit_card', 'last_activity', 'female', 'male', 'equity' 
    ]

# визуализация
row, col, num_cols = 0, 0, 2
fig, myplot = plt.subplots(figsize = (15, 24), nrows = len(features_1), ncols = num_cols)

for u in features_1:
    for churn in [0, 1]:
        col = churn
        sns.barplot(
            x = df.loc[df['churn']==churn, u].value_counts().index, 
            y = df.loc[df['churn']==churn, u].value_counts(), 
            ax  = myplot[row, col]
        )
        myplot[row, col].set_xlabel("")
        myplot[row, col].set_title(u, fontsize = 15)
        if churn == 0:
            myplot[row, col].set_ylabel("Отток (те, кто ушел)")
        else:
            myplot[row, col].set_ylabel("Оставшиеся клиенты")
    row = row + 1

plt.subplots_adjust(hspace = 0.5)
plt.subplots_adjust(wspace = 0.3)
for v in range(row):
    for z in range(num_cols):
        for patch in myplot[v,z].patches:
            label_x = patch.get_x() + patch.get_width()/2  
            label_y = patch.get_y() + patch.get_height()/2
            myplot[v,z].text(label_x, label_y, 
                             str(int(patch.get_height())) + '('+'{:.0%}'.format(
                               patch.get_height()/len(df.loc[df['churn']==z]))+')',
                            horizontalalignment='center', verticalalignment='center')

**Вывод:**

*Общие данные*

|                        |products                          | credit card         |last_activity |
| ---------------------- | -------------------------------- |---------------------|--------------|
|Отток (ушедшие клиенты) | 2 продукта (51%); 1 продукт (38%)|имеют кред. карту 71%|активны 48%   |
|Оставшиеся клиенты      | 2 продукта (54%); 1 продукт (13%)|имеют кред. карту 55%|активны 70%   |


*Гендерное распределение*

|                        |female                  |  male                  |
| ---------------------- | ---------------------- |------------------------|
|Отток (ушедшие клиенты) |53% жен среди ушедших   |47% муж среди ушедших   |
|Оставшиеся клиенты      |35% жен среди оставшихся|65% муж среди оставшихся|

**Итого определено, что в отток ушли:**
- преимущественно женщины;
- клиенты, имеющие кредит.карту;
- клиенты, которые имеют количество баллов собственности = 0  

Первые 3 признака отточных сегментов определены. 

In [ ]:
# список числовых признаков
features_nonbool = ['score', 'age', 'balance', 'est_salary'] 

In [ ]:
# визуализация распределения числовых признаков
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
axes_list = [item for sublist in axes for item in sublist]
fig.tight_layout(pad=5.0)
fig.suptitle('Распределение числовых признаков', fontsize=16)
for feature in features_nonbool:  # цикл
    ax = axes_list.pop(0)
    sns.histplot(df.query('churn == 0')[feature], bins=30, ax=ax, kde=False, color='orange', stat="density",
                 common_norm=False, label='Оставшиеся клиенты')
    sns.histplot(df.query('churn == 1')[feature], bins=30, ax=ax, kde=False, color='skyblue', stat="density",
                 common_norm=False, label='Отток (те, кто ушел)')
    ax.set_xlabel(feature, fontsize=11)
    ax.set_xlim(left=df[feature].min(), right=df[feature].max()) # установка границ оси x по минимальному и максимальному значению признака
    ax.set_ylabel('Кол-во записей', fontsize=12)
    ax.legend()


In [ ]:
# отдельно посмотрим распределение параметра оценочный доход клиента (зар.плата)
fig, ax = plt.subplots(figsize=(7, 7))
fig.tight_layout(pad=5.0)
fig.suptitle('Распределение параметра оценочный доход клиента (зар.плата)', fontsize=16)
sns.histplot(df.query('churn == 0')['est_salary'], bins=30, ax=ax, kde=False, color='orange', stat="density",
             common_norm=False, label='Оставшиеся клиенты')
sns.histplot(df.query('churn == 1')['est_salary'], bins=30, ax=ax, kde=False, color='skyblue', stat="density",
             common_norm=False, label='Отток (те, кто ушел)')
ax.set_xlabel('est_salary', fontsize=11)
ax.set_xlim(left=df['est_salary'].min(), right=600000)
ax.set_ylabel('Кол-во записей', fontsize=12)
ax.legend();


In [ ]:
# отдельно посмотрим распределение параметра баланс на счете клиента
fig, ax = plt.subplots(figsize=(7, 7))
fig.tight_layout(pad=5.0)
fig.suptitle('Распределение параметра баланс на счете клиента', fontsize=16)
sns.histplot(df.query('churn == 0')['balance'], bins=30, ax=ax, kde=False, color='orange', stat="density",
             common_norm=False, label='Оставшиеся клиенты')
sns.histplot(df.query('churn == 1')['balance'], bins=30, ax=ax, kde=False, color='skyblue', stat="density",
             common_norm=False, label='Отток (те, кто ушел)')
ax.set_xlabel('balance', fontsize=11)
ax.set_xlim(left=df['balance'].min(), right=10000000)
ax.set_ylabel('Кол-во записей', fontsize=12)
ax.legend();


**Вывод:**

*Общие данные*

|           |Оставшиеся клиенты                                                  |  Отток (ушедшие клиенты)                                                   |
| ----------| ------------------------------------------------------------------ |----------------------------------------------------------------------------|
|score      |выходит на "плато" к 800 ед. больше всего в диапазоне от 800 до 950.| наблюдается пик на 850 - 870 ед. больше всего в диапазоне от 800 до 950.   |
|age        |больше всего клиентов от 35 до 45 лет                               | больше всего клиентов от 25-35, 37-45, 50-60                                      |
|equity     |больше всего клиентов с 0 кол-вом, далее 2й пик - 4 ед.             | среди ушедших клиентов больше всего состоятельных кол-вом баллов от 4 до 5 |
|balance    |много клиентов с небольшим балансом, вкл. пропуска, далее спад      | видимо у ушедших в отток больше всего пропусков                            |
|est_salary |нет сильного различия в уровне дохода у тех кто ушел в отток и тех, кто остался. гистограммы весьма схожи в обеих группах                        |



**Наиболее очевидно, что в отток ушли состоятельные клиенты, с количеством баллов кредитного скорринга от 800 до 950. Возраст от 25 до 35 лет - самая значительная группа и от 50 до 60 вторая по значимости группа клиентов** - вероятно определены 3й и 4й признак отточного сегмента

### Средние значения относительно оттока 

In [ ]:
# Построим гистограммы чтобы посмотреть средний уровень оттока по признакам
bools = ['equity', 'products', 'rubinsk', 'yaroslavl', 'rostov', 'credit_card', 'last_activity', 'gender_origin'] 

rows = 4
fig, ax = plt.subplots(nrows=rows, ncols=2, figsize=(15, 24))
ax = ax.flatten()

for i in range(len(bools)):
    sns.barplot(x=df[bools[i]], y='churn', data=df, ax=ax[i])
    ax[i].axhline(y=df['churn'].mean(), c='red', label='Средний показатель оттока') 
    ax[i].set_title(f'{bools[i]}')
    ax[i].set_xlabel('')

plt.show()

**Выводы:**

1) Выше всего отток клиентов в Ярославле, далее в Ростове, после Рыбинск, что в целом подтверждает полученную ранее информацию.

2) Больше отток среди клиентов с высоким количеством баллов собственности и несколькими банковскими продуктами, что в целом объяснимо: люди более заинтересованы в финансовой сфере и в поиске оптимальных банковских продуктов;

### Корреляция признаков

In [ ]:
#sns.set(font_scale=1.15)
plt.figure(figsize=(15,10))
sns.heatmap(
    df.corr(), #вычисляем матрицу корреляций        
    cmap='RdBu_r', # задаём цветовую схему
    annot=True, # рисуем значения внутри ячеек
    vmin=-1, vmax=1); # указываем начало цветовых кодов от -1 до 1.

**Выводы:**
1) наиболее сильная положительная кореляция между:
- количеством баллов собственности и количеством продуктов банка ('equity' и 'products') = 0,43: чем состоятельнее клиент, тем больше у него банковскких продуктов;
- уровнем дохода клеиента и балансом его счета ('est_salary' и 'balance') = 0,34: чем больше доход, тем больше счет в банке;
- количеством баллов собственности и оттоком ('equity' и 'churn') = 0,27: получается, что чем больше собственности у клинета банка, тем выше вероятность его оттока.
2) наиболее значимая отрицательная кореляция между:
- наличие/отсутствие кредитной карты и количеством продуктов, которыми пользуется клиент ('credit_card' и 'products') = -0,25;

### Проверка гипотез:

Ранее определено, что на отток влияют:
1) сумма дохода клиента - необходимо проверить гипотезу;
2) количество баллов собственности - необходимо проверить гипотезу;
3) количество продуктов банка - необходимо проверить гипотезу;
4) наличие/отсутствие кредитной карты - необходимо проверить гипотезу;

Проверим применимость t-test (критерия Стьюдента). Именно он применяется для проверки гипотез о равенстве средних значений двух совокупностей - ему отдается приоритет чаще всего. 
Условия применения статистического критерия Стьюдента:
* Генеральные совокупности не должны зависеть друг от друга;
* Выборочные средние должны быть нормально распределены (благодаря ЦПТ, если размер выборки составляет хотя бы несколько десятков значений, выборочные средние, которые можно получить из одной и той же генеральной совокупности, будут распределены нормально вокруг истинного среднего этой совокупности. Это утверждение верно, даже если сама генеральная совокупность не распределена нормально - наш случай);
* Дисперсии рассматриваемых генеральных совокупностей должны быть равны (допустим, что дисперсии у генеральных совокупностей разные, следовательно, укажим equal_var= False при вызове метода scipy.stats.ttest_ind().

#### Гипотеза №1: Различие суммы дохода оставшихся и отточных клиентов

Сформулируем нулевые гипотезы:

    - H₀: средние доходы оставшихся и отточных клиентов равны
    - H₁: средние доходы оставшихся и отточных клиентов не равны

In [ ]:
#посчитаем среднюю оценку для гипотезы №1
print('Средняя оценка по доходу клиентов не в оттоке', df[df['churn'] == 0]['est_salary'].mean())
print('Средняя оценка по доходу клиентов в оттоке', df[df['churn'] == 1]['est_salary'].mean())

In [ ]:
# проверка с помощью теста Левена
# тест Левана с определением медианы

test_leven, p = st.levene(df[df['churn'] == 0]['est_salary'], df[df['churn'] == 1]['est_salary'], center='median')
print('Statistics=%.3f, p-value=%.3f' % (test_leven, p))
alpha = 0.05
if p < alpha:
    print('Отклонить гипотезу о равенстве дисперсий')
else:
    print('Принять гипотезу о равенстве дисперсий')

In [ ]:
# проверка с помощью теста Левена
#тест Левана с определением среднего значения

test_leven, p = st.levene(df[df['churn'] == 0]['est_salary'], df[df['churn'] == 1]['est_salary'], center='mean')
print('Statistics=%.3f, p-value=%.3f' % (test_leven, p))
alpha = 0.05
if p < alpha:
    print('Отклонить гипотезу о равенстве дисперсий')
else:
    print('Принять гипотезу о равенстве дисперсий')

**Вывод:** 
Тест Левина проверяет нулевую гипотезу о том, что дисперсии равны. По результатом проверки p-value меньше 0,05, следовательно, мы не можем подтвердить нулевую гипотезу и вправе сделать вывод о наличии разницы между дисперсиями.

In [ ]:
# посмотрим гистограмму распределения
df[df['churn'] == 0]['est_salary'].hist(bins=30, alpha=0.5)
df[df['churn'] == 1]['est_salary'].hist(bins=30, alpha=0.5)
plt.title('Гистограмма распределения')
plt.xlabel('оценочный доход клиента (est_salary)')
plt.ylabel('Кол-во записей');

Применим U-критерий Манна-Уитни для сравнения различий между двумя выборками, так как было установлено, что распределение не является нормальным (результаты тестов + длинный правый хвост на гистограмме)

In [ ]:
# применим тест Манна-Уитни
stat, p_value = mannwhitneyu(df[df['churn'] == 0]['est_salary'], df[df['churn'] == 1]['est_salary']) 
print('Statistics=%.2f, p=%.2f' % (stat, p_value)) 
# установим уровень значимости
alpha = 0.05
# выводы
if p_value < alpha: 
    print('Отклонить нулевую гипотезу') 
else: 
    print('Не получилось отклонить нулевую гипотезу')

**Вывод:** Отклоняем нулевую гипотезу, то есть средние доходы оставшихся и отточных клиентов не равны.

#### Гипотеза №2: Pазличие количества баллов собственности оставшихся и отточных клиентов

Сформулируем нулевые гипотезы:

    - H₀: средняя оценка баллов собственности оставшихся и отточных клиентов равны
    - H₁: средняя оценка баллов собственности оставшихся и отточных клиентов не равны

In [ ]:
#посчитаем среднюю оценку для гипотезы №2
print('Средняя оценка по доходу клиентов не в оттоке', df[df['churn'] == 0]['equity'].mean())
print('Средняя оценка по доходу клиентов в оттоке', df[df['churn'] == 1]['equity'].mean())

In [ ]:
# посмотрим гистограмму распределения
df[df['churn'] == 0]['equity'].hist(bins=30, alpha=0.5)
df[df['churn'] == 1]['equity'].hist(bins=30, alpha=0.5)
plt.title('Гистограмма распределения')
plt.xlabel('Количество баллов собственности (equity)')
plt.ylabel('Кол-во записей');

In [ ]:
# проверка с помощью теста Левена
# тест Левана с определением медианы

test_leven, p = st.levene(df[df['churn'] == 0]['equity'], df[df['churn'] == 1]['equity'], center='median')
print('Statistics=%.3f, p-value=%.3f' % (test_leven, p))
alpha = 0.05
if p < alpha:
    print('Отклонить гипотезу о равенстве дисперсий')
else:
    print('Принять гипотезу о равенстве дисперсий')

In [ ]:
# проверка с помощью теста Левена
# тест Левана с определением среднего значения

test_leven, p = st.levene(df[df['churn'] == 0]['equity'], df[df['churn'] == 1]['equity'], center='mean')
print('Statistics=%.3f, p-value=%.3f' % (test_leven, p))
alpha = 0.05
if p < alpha:
    print('Отклонить гипотезу о равенстве дисперсий')
else:
    print('Принять гипотезу о равенстве дисперсий')

**Вывод:** Отклоняем гипотезу о равенстве дисперсий.

Установлено, что 2х выборок неравные дисперсии, приемним t-тест Уэлча. Он рассматривается как параметрический эквивалент T-теста с двумя выборками. Так же t-критерий Уэлча работает лучше, чем t-критерий Стьюдента, когда размеры выборки и дисперсии между группами не равны, и дает идентичные результаты, когда размеры выборки дисперсии равны.

In [ ]:
# results = вызов метода для проверки гипотезы

results = st.ttest_ind (
    df[df['churn']==0]['equity'],
    df[df['churn']==1]['equity'], equal_var = False)

alpha = 0.05

# вывод значения p-value на экран 
print (results.pvalue) 
# условный оператор с выводом строки с ответом

if results.pvalue < alpha:
    print ('Отвергаем нулевую гипотезу')
else:
    print ('Не получилось отвергнуть нулевую гипотезу')

**Вывод:** Отвергаем нулевую гипотезу. Средняя оценка баллов собственности оставшихся и отточных клиентов не равны; в целом ранее установлено, что более состоятельные клиенты (большее кол-во баллов собственности) более склонны к оттоку.

#### Гипотеза №3: Pазличие количества продуктов банка оставшихся и отточных клиентов

Сформулируем нулевые гипотезы:

    - H₀: среднее количество продуктов банка оставшихся и отточных клиентов равны
    - H₁: среднее количество продуктов банка оставшихся и отточных клиентов не равно

In [ ]:
#посчитаем среднюю оценку для гипотезы №3
print('Средняя оценка по доходу клиентов не в оттоке', df[df['churn'] == 0]['products'].mean())
print('Средняя оценка по доходу клиентов в оттоке', df[df['churn'] == 1]['products'].mean())

In [ ]:
# посмотрим гистограмму распределения
df[df['churn'] == 0]['products'].hist(bins=30, alpha=0.5)
df[df['churn'] == 1]['products'].hist(bins=30, alpha=0.5)
plt.title('Гистограмма распределения')
plt.xlabel('Количество продуктов банка (products)')
plt.ylabel('Кол-во записей');

In [ ]:
# проверка с помощью теста Левена
#тест Левана с определением среднего значения

test_leven, p = st.levene(df[df['churn'] == 0]['products'], df[df['churn'] == 1]['products'], center='mean')
print('Statistics=%.3f, p-value=%.3f' % (test_leven, p))
alpha = 0.05
if p < alpha:
    print('Отклонить гипотезу о равенстве дисперсий')
else:
    print('Принять гипотезу о равенстве дисперсий')

Установлено, что 2х выборок неравные дисперсии, приемним t-тест Уэлча. Он рассматривается как параметрический эквивалент T-теста с двумя выборками. Так же t-критерий Уэлча работает лучше, чем t-критерий Стьюдента, когда размеры выборки и дисперсии между группами не равны, и дает идентичные результаты, когда размеры выборки дисперсии равны.

In [ ]:
# results = вызов метода для проверки гипотезы

results = st.ttest_ind (
    df[df['churn']==0]['products'],
    df[df['churn']==1]['products'], equal_var = False)

alpha = 0.05

# вывод значения p-value на экран 
print (results.pvalue) 
# условный оператор с выводом строки с ответом

if results.pvalue < alpha:
    print ('Отвергаем нулевую гипотезу')
else:
    print ('Не получилось отвергнуть нулевую гипотезу')

**Вывод:** Отвергаем нулевую гипотезу. Среднее количество продуктов банка оставшихся и отточных клиентов не равно; в целом ранее установлено, что более состоятельные клиенты (большее кол-во баллов собственности) более склонны к оттоку.

#### Гипотеза №4: Pазличие количества кредитных карт банка оставшихся и отточных клиентов

Сформулируем нулевые гипотезы:

    - H₀: среднее количество кредитных карт банка оставшихся и отточных клиентов равны
    - H₁: среднее количество кредитных карт банка оставшихся и отточных клиентов не равно

In [ ]:
#посчитаем среднюю оценку для гипотезы №4
print('Средняя оценка по доходу клиентов не в оттоке', df[df['churn'] == 0]['credit_card'].mean())
print('Средняя оценка по доходу клиентов в оттоке', df[df['churn'] == 1]['credit_card'].mean())

In [ ]:
# посмотрим гистограмму распределения
df[df['churn'] == 0]['credit_card'].hist(bins=30, alpha=0.5)
df[df['churn'] == 1]['credit_card'].hist(bins=30, alpha=0.5)
plt.title('Гистограмма распределения')
plt.xlabel('Наличие кредитной карты (credit_card)')
plt.ylabel('Кол-во записей');

In [ ]:
# проверка с помощью теста Левена
#тест Левана с определением среднего значения

test_leven, p = st.levene(df[df['churn'] == 0]['credit_card'], df[df['churn'] == 1]['credit_card'], center='mean')
print('Statistics=%.3f, p-value=%.3f' % (test_leven, p))
alpha = 0.05
if p < alpha:
    print('Отклонить гипотезу о равенстве дисперсий')
else:
    print('Принять гипотезу о равенстве дисперсий')

In [ ]:
# проверка с помощью теста Левена
#тест Левана с определением медианы

test_leven, p = st.levene(df[df['churn'] == 0]['credit_card'], df[df['churn'] == 1]['credit_card'], center='median')
print('Statistics=%.3f, p-value=%.3f' % (test_leven, p))
alpha = 0.05
if p < alpha:
    print('Отклонить гипотезу о равенстве дисперсий')
else:
    print('Принять гипотезу о равенстве дисперсий')

**Вывод:** отклоняем гипотезу о равенстве дисперсий.

Так как распределение не явл.нормальным и размер выборки большой применим Z-тест 

In [ ]:
# применим z-тест
#определим размер групп
group_size = df['credit_card'].value_counts(sort=False) 

#определеим размер целевых подгрупп в группах
share_size = df.groupby('credit_card')['churn'].sum()

z_value, p_value = proportions_ztest(share_size, group_size)

test_leven, p = st.levene(df[df['churn'] == 0]['credit_card'], df[df['churn'] == 1]['credit_card'], center='median')
print('Statistics=%.3f, p-value=%.3f' % (z_value, p))
alpha = 0.05
if p < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу, статистически значимых различий в выборках нет')

In [ ]:
# без округления
z_value

In [ ]:
# без округления
p_value

**Вывод:** Отвергаем нулевую гипотезу. Как ранее было установлено в ходе анализа среди клиентов, ушедших в отток 71% пользователей кредитной картой, а среди оставшихся 55%.

### Выводы по этапу Исследовательский анализ данных (EDA)

<div style="border:solid green 2px; padding: 20px">
    
1) по количеству банковских продуктов:(top3)
   - 51% клиентов имеет 2 банковских продукта;
   - 33% клиентов имеет 1 банковский продукт;
   - 10% клиентов имеет 3 банковских продукта;
    
2) кредитная карта есть у 68% клиентов;
    
3) активность клиентов:
  - 52% активны;
  - 48% не активны;
    
4) **в отток ушло 18% клиентов**;
    
5) гендерное распределение среди клиентов 50/50;
    
6) распределение клиентов по городам: Ростов (14%), Рыбинск (27%), Ярославль (59%)
    
7) по географическому признаку: больше всего ушло клиентов из Ярославля (19,02%); Ростов (18,77%); Рыбинск (16,18%)


  *Общие данные*

|                        |products                          | credit card             |last_activity |
| ---------------------- | -------------------------------- |-------------------------|--------------|
|Отток (ушедшие клиенты) | 2 продукта (51%); 1 продукт (38%)|**имеют кред. карту 71%**|активны 48%   |
|Оставшиеся клиенты      | 2 продукта (54%); 1 продукт (13%)|имеют кред. карту 55%    |активны 70%   |


*Гендерное распределение*

|                        |female                    |  male                  |
| ---------------------- | ------------------------ |------------------------|
|Отток (ушедшие клиенты) |**53% жен среди ушедших** |47% муж среди ушедших   |
|Оставшиеся клиенты      |35% жен среди оставшихся  |65% муж среди оставшихся|


*Географическое распределение*

|                        |  Ярославль  |Ростов      |  Рыбинск |
|----------------------- | ------------| -----------|----------|
| Кол-во клиентов        |5 874        |1 417       |2 694     |
| Доля ушедших клиентов  |19.02%       |18.77%      |16.18%    |


*Результат анализа исследования распределения числовых признаков*

|           |Оставшиеся клиенты                                                  |  Отток (ушедшие клиенты)                                                   |
| ----------| ------------------------------------------------------------------ |----------------------------------------------------------------------------|
|score      |выходит на "плато" к 800 ед. больше всего в диапазоне от 800 до 950.| наблюдается пик на 850 - 870 ед. больше всего в диапазоне от 800 до 950.   |
|age        |больше всего клиентов от 35 до 45 лет                               | больше всего клиентов от 25-35, 37-45, 50-60                                      |
|equity     |больше всего клиентов с 0 кол-вом, далее 2й пик - 4 ед.             | среди ушедших клиентов больше всего состоятельных кол-вом баллов от 4 до 5 |
|balance    |много клиентов с небольшим балансом, вкл. пропуска, далее спад      | видимо у ушедших в отток больше всего пропусков                            |
|est_salary |нет сильного различия в уровне дохода у тех кто ушел в отток и тех, кто остался. гистограммы весьма схожи в обеих группах   

<div style="border:solid green 2px; padding: 20px">

*Выдвинутые и проверенные гипотезы:*
    
Гипотеза №1:
- H₀: средние доходы оставшихся и отточных клиентов равны (отвергаем);
- H₁: средние доходы оставшихся и отточных клиентов не равны;
    
Гипотеза №2:
- H₀: средняя оценка баллов собственности оставшихся и отточных клиентов равны (отвергаем);
- H₁: средняя оценка баллов собственности оставшихся и отточных клиентов не равны;
    
Гипотеза №3:
- H₀: среднее количество продуктов банка оставшихся и отточных клиентов равны (отвергаем);
- H₁: среднее количество продуктов банка оставшихся и отточных клиентов не равны;

Гипотеза №4:
- H₀: среднее количество кредитных карт банка оставшихся и отточных клиентов равны (отвергаем);
- H₁: среднее количество кредитных карт банка оставшихся и отточных клиентов не равны

## Сегментация

По итогам проведенного анализа и проверки гипотез выделим несколько высокоотточных сегментов клиентов банка. При этом в качестве критериев используем следующие параметры:

* пол ('gender') или прекодированные столбцы ('f', 'm');
* возраст ('age');
* количество баллов собственности ('equity');
* количество продуктов банка ('products');
* оценочный доход клиента ('est_salary');
* активность клиента ('last_acrtivity')

In [ ]:
# напишем функцию для сегментации
def get_segment(row):
    result = [0, 0, 0]
    if row['male'] == 1 and row['balance']>= 80000:
        result[0]  = 1
    if row['products'] >= 2 and row['score']>= 700:
        result[1]  = 1
    if row['est_salary'] >= 50000 and row['last_activity'] == 1:
        result[2]  = 1
#     else:
#         return 'клиенты, не попавшие в сегментацию'
    return np.array(result)

In [ ]:
df[['segment_1', 'segment_2', 'segment_3']] = np.vstack(df.apply(get_segment, axis=1))

In [ ]:
# Построим гистограммы чтобы посмотреть средний уровень оттока по сегментам
list_segment = [
    'segment_1', 'segment_2', 'segment_3' 
    ] 

rows = 2
fig, ax = plt.subplots(nrows=rows, ncols=2, figsize=(10, 12))
ax = ax.flatten()

for i in range(len(list_segment)):
    sns.barplot(x=df[list_segment[i]], y='churn', data=df, ax=ax[i])
    ax[i].axhline(y=df['churn'].mean(), c='red', label='Средний показатель оттока') 
    ax[i].set_title(f'{list_segment[i]}')
    ax[i].set_xlabel('')

plt.show()

**Вывод:** средний уровень оттока по сегментам выше, чем средний уровень оттока по банку в целом (18%) 

In [ ]:
# посмотим кол-во клиентов банка в каждом сегменте
df[['segment_1', 'segment_2', 'segment_3']].sum(axis=0)

In [ ]:
# посмотрим средний отток в сегменте_1
df.loc[df['segment_1']==1, 'churn'].mean()

In [ ]:
# посмотрим средний отток в сегменте_2
df.loc[df['segment_2']==1, 'churn'].mean()

In [ ]:
# посмотрим средний отток в сегменте_3
df.loc[df['segment_3']==1, 'churn'].mean()

**Вывод:** количество клиентов банка в сегментах более 500 человек, средний %оттока выше 18% (средний %оттока по банку)

### Приоритизация сегментов

Приоритезируем сегменты по мере убывания отточных клиентов:

1. Сегмент №1: 3 551 чел (мужчины с балансом на счету более 80 000) - 32,07 % оттока;
2. Сегмент №3: 4 537 чел (активные клиенты с оценочным доходом (зар.плата) более 50 000) - 25,85 % оттока;
3. Сегмент №2: 6 621 чел (клиенты банка имеющие более 2х продуктов банка и кредитный скоринг более 700) - 23,90 % оттока.

<div style="border:solid green 2px; padding: 20px">

## Рекомендации:

Сегмент 1: мужчины с балансом на счету более 80 000 (3 551 человек, 32,07 % оттока):
    
    - возможность самостоятельно выбирать категории кэшбэка (доплатить за выбор конткретной категории на месяц);    
    - среди программ лояльности предлагать больше автомобильной тематики (кэшбэк за бензин), программ связанных с недвижимостью;

    
Сегмент 3: активные клиенты с оценочным доходом (зар.плата) более 50 000 (4 537 человек, 25,85 % оттока):
    
    - более высокий % на накопительном счету; 
    - предложить выгодный перевод з/п в банк «Метанпромбанк» (в случае получения з/п в др. банке)
    - информировать о новых доступных банковских продуктах;
    
Сегмент 2: клиенты банка имеющие более 2х продуктов банка и кредитный скоринг более 700 (6 621 человек, 20,90 % оттока):
    
    - предложить специальные условия по страхованию объектов собственности (авто, недвижимость);    
    - кредитная карта с большей суммой кредита/более длительным сроком погашения кредита;
    - возможность самостоятельно выбирать категории кэшбэка (доплатить за выбор конткретной категории на месяц);
    - информировать о новых доступных банковских продуктах;
    